In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 뉴럴 스타일 트랜스퍼
- 스타일 트랜스퍼는 참조 이미지의 스타일을 적용하면서 타깃 이미지의 콘텐츠를 보존하여 새로운 이미지를 만드는 방법
- 타깃 이미지의 콘텐츠를 보존하면서 참조 이미지의 스타일을 타깃 이미지에 적용
- 스타일은 질감, 색깔, 이미지에 있는 다양한 크기의 시각 요소를 의미
- 콘텐츠는 이미지에 있는 고수준의 대형 구조를 의미
- 사전 훈련된 컨브넷으로 손실을 정의하고 이 손실을 최적화하는 과정으로 스타일 트랜스퍼를 구성
- 다양한 변종과 개선이 가능

**스타일 이미지와 콘텐츠 이미지 준비하기**

In [ ]:
from tensorflow import keras

base_image_path = keras.utils.get_file(
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

943128/943128 [==============================] - 0s 0us/step


**유틸리티 함수**

In [ ]:
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

**사전 훈련된 VGG19 모델을 사용해 특성 추출기 만들기**

In [ ]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80134624/80134624 [==============================] - 0s 0us/step


**콘텐츠 손실**

In [ ]:
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

**스타일 손실**

In [ ]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

**총 변위 손실**

In [ ]:
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

**최소화할 최종 손실 정의하기**

In [ ]:
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

**경사 하강법 단계 설정하기**

In [ ]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"{i}번째 반복: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

100번째 반복: loss=8136.38
200번째 반복: loss=6651.94
300번째 반복: loss=6062.30
400번째 반복: loss=5741.20
500번째 반복: loss=5535.10
600번째 반복: loss=5389.86
700번째 반복: loss=5280.72
800번째 반복: loss=5195.17
900번째 반복: loss=5126.04
1000번째 반복: loss=5069.00
1100번째 반복: loss=5021.12
1200번째 반복: loss=4980.28
1300번째 반복: loss=4945.01
1400번째 반복: loss=4914.24
1500번째 반복: loss=4887.16
1600번째 반복: loss=4863.07
1700번째 반복: loss=4841.59
1800번째 반복: loss=4822.24
1900번째 반복: loss=4804.82
2000번째 반복: loss=4789.02
2100번째 반복: loss=4774.66
2200번째 반복: loss=4761.51
2300번째 반복: loss=4749.40
2400번째 반복: loss=4738.24
2500번째 반복: loss=4727.98
2600번째 반복: loss=4718.51
2700번째 반복: loss=4709.76
2800번째 반복: loss=4701.61
2900번째 반복: loss=4694.03
3000번째 반복: loss=4686.96
3100번째 반복: loss=4680.36
3200번째 반복: loss=4674.20
3300번째 반복: loss=4668.41
3400번째 반복: loss=4662.98
3500번째 반복: loss=4657.89
3600번째 반복: loss=4653.12
3700번째 반복: loss=4648.62
3800번째 반복: loss=4644.39
3900번째 반복: loss=4640.42
4000번째 반복: loss=4636.67


![combination_image_at_iteration_4000.png](https://github.com/rickiepark/deep-learning-with-python-2nd/blob/main/combination_image_at_iteration_4000.png?raw=1)